Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

***

# Graded Assignment General Overview

During the course, we will have three graded assignments. In this three assignment, you will go through the entire Probabilistic Programming (PP) workflow we cover in the course. In the first assignment, we will start with the first three steps of the workflow; assignment two will cover steps 4 and 5, and assignment three will cover step 6:

|Assignment|Workflow Step|
|---|---|
|Assignment 1|1. Quantity to estiamte|
|            |2. Scientific Modele|
|            |3. Statistical Model|
|Assignment 2|4. Synthetic Data|
|            |5. Testing|
|Assignment 3|6. Real data|


__Background Paper__

You will implement the workflow for a specific research project. We will use an existing paper as a basis (see table). While the paper is important for background information, in most parts, the assignment will be based on the preregistration that belongs to the paper. A preregistration is a document that outlines the research question, survey design, and estimation approach before the actual experiment is conducted. The data used in the paper is also publicly available, but there is no need to download it now; this will be done within the assignments.

|Resource|Title|Link|
|-----------|---------|---------|
|Paper |Zachmann, L., McCallum, C., & Finger, R. (2023). Nudging farmers towards low‐pesticide practices: Evidence from a randomized experiment in viticulture. Journal of the Agricultural and Applied Economics Association, 2(3), 497-514.|[https://doi.org/10.1002/jaa2.76](https://doi.org/10.1002/jaa2.76) |
|Preregistration|Zachmann, L., McCallum, C., & Finger, R. 'Uptake of fungi-resistant grapevines: The effect of personalized vs general risk exposure information'. Pre-registered on 01/12/2022, AsPredicted #84972.| [https://aspredicted.org/fb2c9.pdf](https://aspredicted.org/fb2c9.pdf) |
|Open Data|Data on Swiss grapevine growers’ production, pest management and risk management decisions| [https://doi.org/10.3929/ethz-b-000568595](https://doi.org/10.3929/ethz-b-000568595) |

__Aim of the Assignments__ 

The paper does not use a PP workflow but uses frequentist estimation methods. We will rebuild the entire research process using the PP workflow and estimate the quantities of interest using Bayesian methods. The paper conducts a survey with farmers to gather the data required for answering the research question. This could be similar to what you could do for your master's thesis. By working on the assignment, you practice how the PP workflow can be implemented for an actual research project. Additionally, it prepares you for applying it to your own research project.    

*Heads up!* Be prepared that this will be quite a challenge and that for most of the questions there is not a single or clear-cut answer. Actually, an important learning outcome is that on the way to answering the research question, we need to make many (subjective) choices, and many alternative specifications or options would also be possible and justified. In terms of the grading, this means that the aim is not to find *one* correct answer (which does not exist) but rather to document and explain that you have thought about your choices and are aware of the implications.

***

# Assingment 2

__Structure of assignement 2__

1) Implement the statistical model in code 

2) Test the model inference using synthetic data

3) Prepare output illustration that allow testing the hypothesis we aim to test


*Good luck!*


***

__Install and import packages__

***

In [ ]:
# Install required packaged (only need to be done once after each restart of the jupyter server)
#!pip install -q pandas matplotlib seaborn numpyro==0.15.2 jax==0.4.26 jaxlib==0.4.26 arviz networkx daft
!pip install -q pandas matplotlib seaborn numpyro==0.15.2 jax jaxlib arviz networkx daft
%conda install python-graphviz # install graphviz using conda (seems not to work with pip)
from IPython.display import clear_output
clear_output(wait=False) # clear all the output from conda

In [ ]:
# Required imports
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from graphviz import Digraph

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import arviz as az
import jax 
from jax import random
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive
import numpyro.optim as optim
from numpyro.infer import SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoLaplaceApproximation
from numpyro.diagnostics import print_summary

# Set seed for reproducibility
rng_key = random.PRNGKey(1)
np.random.seed(0)

***

__*Preparation*: Generate data to mirror the survey structure described in the preregistration__

***

In [ ]:
# Generate data to mirror the data structure in Zachmann
n = 294  # number of targeted in preregistration
# Randomly assign treatment groups (g=0 control, g=1 personalized, g=2, general)
G = np.random.randint(0,3,n) 
print('Number of farmers',n)
print('G.shape',G.shape)

# Plot the assignment to treatment
plt.bar(*np.unique(G,return_counts=True));
plt.ylabel('counts');
plt.xticks([0,1,2], labels=['control','personalized','general']);

***

__*Preparation*: Define the DAG and statistical model to have the same starting point__

***

Note: In order for everybody to have the same starting point for this second assignment, we all work with the same DAG and statistical model. In principle, you could continue with your model defined in the first assignment. However, that would make follow-up questions and grading difficult to align. The model provided below is very similar to the specification in the paper. However, this does not mean that this is the best or the only correct solution. In fact, there are multiple ways to improve the model. However, the model is relatively simple, making the following steps not too complicated. In general, it is a good strategy to start with a very simple model and go through the entire process; once this is running, we could stepwise make the model better. 


***

In [ ]:
# Define a DGP as in assignment 01 
dot = Digraph()

# Observed notes
dot.attr('node', style='filled', color='white') # Set color to white
dot.node('Tper', 'Personal info')
dot.node('Tgen', 'General info')
dot.node('ES', 'Delta E[S]')
dot.node('StdS', 'Delta Std[S]')

dot.edges([('Tper','ES'),('Tgen','ES'),
          ('Tper','StdS'),('Tgen','StdS')])
dot

__Based on the DAG define a complete data generating process (DGP), i.e. statistical model__ 

Let $G=0,1,2$ denote our three treatment groups, with $g=0$ being the control group, $g=1$ being the personalized info group, and $g=2$ being the generalized info group. Further, let $G[i]$ denote indexing the group of observation $i$. 

Define $Y_1$: Change in expected share of fungi-resistant grapevines (change before/after information treatment). The expectation is derived by asking for the most likely proportion, the minimum, and the maximum proportion. Then a triangular share distribution is used to approximate the expectation. 

*Model for $\Delta E(S_i)$:*

$Y_{1i} = \Delta E(S_i) \sim Normal(\theta^{E}_{G[i]},\sigma^{E})  $

$\theta^E_g \sim Normal(0,0.3) $  for $g=0,1,2 $

$\sigma^{E} \sim Exponential(3) $

*Model for $\Delta Std(S_i)$:*

Define $Y_2$: Change in the standard deviation (std) of the intended share of fungi-resistant grapevines (change before/after information treatment). The std is also derived from the triangular share distribution. 

$Y_{2i} = \Delta Std(S_i) \sim Normal(\theta^S_{G[i]},\sigma^{Std})  $

$\theta^S_g \sim Normal(0,0.3) $  for $g=0,1,2 $

$\sigma^{Std} \sim Exponential(3) $

***

## Implement the statistical model in NumPyro code

The next cell implements the statistical model in NumPyro code. Review the code and make sure you understand how it relates to the DGP defined above.

In [ ]:
def modelDiff(G, DiffEI=None,DiffStdI=None):
    # Model for Diff EI
    mu_DiffEI = numpyro.sample('mu_DiffEI', dist.Normal(0,0.3).expand([3]))
    sigma_DiffEI = numpyro.sample('sigma_DiffEI', dist.Exponential(3))
    DiffEI = numpyro.sample('DiffEI', dist.Normal(mu_DiffEI[G],sigma_DiffEI), obs=DiffEI)

    # Model of Diff StdI
    mu_DiffStdI = numpyro.sample('mu_DiffStdI', dist.Normal(0,0.3).expand([3]))
    sigma_DiffStdI = numpyro.sample('sigma_DiffStdI', dist.Exponential(3))
    DiffStdI = numpyro.sample('DiffStdI', dist.Normal(loc=mu_DiffStdI[G], scale=sigma_DiffStdI), obs=DiffStdI)
    
# Sample from prior
rng_key, rng_key_ = random.split(rng_key)
prior_predictive = Predictive(modelDiff, num_samples=100)
prior_samples = prior_predictive(rng_key_,G=G)

print('Elements in "prior_predictive"', prior_samples.keys())

***
<div class="alert alert-block alert-success">

__*Task 1*: Implement the statistical model in NumPyro code__ (6 points)

</div>
a) Use the cell below to print the shape of all the elements in the dictionary "prior_predictive" (3 points)

*For a): Implement you answer using the cell below*

b) Explain in your own words, for every element in the dictionary "prior_predictive"  why it has the respective shape, i.e., define verbally the dimension of each variable. (3 points)

For example, assume a ```a.shape = (3,4)``. You should say what the 3 and the 4 mean. 

*For b): Add your answer here:*

YOUR ANSWER HERE


In [ ]:
# 1a) Print shape of keys
print('Shape of elements in "prior_predictive"', prior_samples.shape)
raise NotImplementedError()

***
## Prior predictive check

<div class="alert alert-block alert-success">

__*Task 2)*: Prior predictive check__ (8 Points)
</div>

Generally, we would need to check all the priors used in the model. For this task, we restrict ourselves to checking only one prior. 

Above we use a prior distribution: 

$\sigma^{E} \sim Exponential(3) $ 

consider as an alternative prior: 

$\sigma^{E} \sim Exponential(1) $ 


2a) Use the cell below to implement this different prior (all else remains the same). (4 Points)

*For a): Implement your answer using the cell below*

b) Use the cell after the next to plot the prior distribution from these two models. Discuss which specification you find more appropriate and motivate why you think so. (4 Points)

*For b): Add your answer here:*

YOUR ANSWER HERE


In [ ]:
# Use this cell to answer Task 2a)
def modelDiff_altPrior(G, DiffEI=None,DiffStdI=None):
    # Model for Diff EI
    # YOUR CODE HERE
    raise NotImplementedError()

    # Model of Diff StdI
    mu_DiffStdI = numpyro.sample('mu_DiffStdI', dist.Normal(0,0.3).expand([3]))
    sigma_DiffStdI = numpyro.sample('sigma_DiffStdI', dist.Exponential(3)) # 2 Points
    DiffStdI = numpyro.sample('DiffStdI', dist.Normal(loc=mu_DiffStdI[G], scale=sigma_DiffStdI), obs=DiffStdI)
    
# Sample from prior
rng_key, rng_key_ = random.split(rng_key)
prior_predictive_altPrior = Predictive(modelDiff_altPrior, num_samples=100)
prior_samples_altPrior = prior_predictive_altPrior(rng_key_,G=G)    
    

In [ ]:
# Use this cell to plot prior samples from the two different model specifications
# Sample from orginal specification 
rng_key, rng_key_ = random.split(rng_key)
prior_predictive = Predictive(modelDiff, num_samples=100)
prior_samples = prior_predictive(rng_key_,G=G)

# Sample from alternative prior specification
rng_key, rng_key_ = random.split(rng_key)
prior_predictive_altPrior = Predictive(modelDiff_altPrior, num_samples=100)
prior_samples_altPrior = prior_predictive_altPrior(rng_key_,G=G)    

def plot_hist_G(outcome,G,xlabel='x',ref=None,ax=None, title=None):
    if ax is None:
        fig,ax = plt.subplots(figsize=(4,4))
    ax.hist(outcome[:,G==0].flatten(),alpha=0.5,label="control",bins=50);
    ax.hist(outcome[:,G==1].flatten(),alpha=0.5,label="pers.",bins=50);
    ax.hist(outcome[:,G==2].flatten(),alpha=0.5,label="gen.",bins=50);
    if title is None:
        ax.set_title('Difference');
    else:
        ax.set_title(title);
        
    ax.set_xlabel(rf'$\Delta$ {xlabel}');
    ax.legend(frameon=False)

    if ref is not None:
        ax.axvline(ref[0],ls='--', color = 'black',)
        ax.axvline(ref[1],ls='--', color = 'black',)
        ax.axvline(ref[2],ls='--', color = 'black',)

fig, (ax1,ax2) = plt.subplots(1,2, tight_layout=True,figsize=(8,4))

plot_hist_G(prior_samples['DiffEI'],G,xlabel='DiffEI',ax=ax1,title='Orignal Prior')
plot_hist_G(prior_samples_altPrior['DiffEI'],G,xlabel='DiffEI',ax=ax2, title='Alternative Prior')

# Note that this restrict the x-axis that is shown to the most relevant range, you might want to comment this out to see the entire distributions
ax1.set_xlim(-3,3);
ax2.set_xlim(-3,3);


***

## Test Model with synthetic data

In a assingment 1 we specified the hypothesis we aim to test. To have the same starting point, assume we would want to test the following hypothesis:

__H1__: Information change intentions, and personalized information on fungicide risk exposure is more important than more general information to change intentions (here, land devoted to fungi-resistant grapes).

We can test the hypothesis by comparing if $\theta^E_1>\theta^E_0$ and $\theta^E_2>\theta^E_0$, i.e. information changes intentions. And $\theta^E_1>\theta^E_2$. i.e. personalized information is more important than more general information.

__H2__: Because personalized information is more relevant for the farmer, personalized information __reduces__ the uncertainty of the intention more then general information.

We can test the hypothesis by comparing if $\theta^S_1<\theta^S_0$, i.e., information reduces uncertainty compared to the control group, and $\theta^S_1<\theta^S_2$, i.e., personalized information leads to a larger reduction in the uncertainty then generalized information.


***
__Condition the model to create a synthetic dataset__

The cell below can be used to condition the coefficient of the model to specific values. We can then use these conditioned model to generate synthetic data where we know exactly
the "true" coefficients. Consider how we did this in examples in the lecture.
    
Using the cell below, we want to generate synthetic data for which all our assumed hypotheses above are true. The general idea is that we then use this synthetic data in our inference step to test whether we can test our hypothesis in a setting where we know that it is true. 

<div class="alert alert-block alert-success">

__*Task 3)*: Condition the model to create a synthetic dataset__ (10 Points)
</div>

2a) Use the cell below to condition the model on parameter values that would exactly match the hypothesis we defined above. (4 Points)

*For a): Implement your answer using the cell below*

b) Motivate and explain your chosen values for each of the six coefficients. Explain how you interpret each of the six coefficients verbally. (6 Points)

*For b): Add your answer here:*

YOUR ANSWER HERE




In [ ]:
# Use this cell to answer 4a)

# Condition the model 
coefTrue = {
            # YOUR CODE HERE
            # 'mu_DiffEI': ... , 
            # 'mu_DiffStdI': ... ,
            'sigma_DiffEI':np.array([0.01]),
            'sigma_DiffStdI':np.array([0.01]),
           }
# Use coefTrue to condition the model
condition_model = numpyro.handlers.condition(modelDiff, data=coefTrue)
# Generate synthetic data using conditioned model
conditioned_predictive = Predictive(condition_model, num_samples=1)
condition_samples = conditioned_predictive(rng_key_,G=G)
condition_samples

# Plot the generated data
fig, (ax1,ax2) = plt.subplots(1,2, tight_layout=True,figsize=(8,4))
plot_hist_G(condition_samples['DiffEI'],G,xlabel='DiffEI',ref=condition_samples['mu_DiffEI'][0],ax=ax1, title='')
plot_hist_G(condition_samples['DiffStdI'],G,xlabel='DiffStdI',ref=condition_samples['mu_DiffStdI'][0],ax=ax2, title='')


In [ ]:
# Use the conditioned data to define the outcome variables 
DiffEI = condition_samples['DiffEI'][0]
DiffStdI = condition_samples['DiffStdI'][0]
print('DiffEI.shape',DiffEI.shape)
print('DiffStdI.shape',DiffStdI.shape)

## Run the model backward (inference), using the conditioned data

The next cell shows the inference step using quadratic approximation (in the commented-out part), which we have used multiple times in class. However, here we use MCMC, even though we have not covered it in class. This is also what you might want to do in an actual application. The next cell gives code for quadratic approximation and for MCMC. For the following, you can basically ignore how we perform inference here. The crucial point is that in base cases, we obtain samples from the posterior; how we get those samples does not really matter.

In [ ]:
#guide = AutoLaplaceApproximation(modelDiff)
#svi = SVI(modelDiff,guide,optim.Adam(0.1),Trace_ELBO(),
#    G=G,DiffEI=DiffEI,DiffStdI=DiffStdI
#)
#svi_result = svi.run(random.PRNGKey(0), 2000)
#params = svi_result.params
#post_samples = guide.sample_posterior(random.PRNGKey(1), params, sample_shape=(1000,))
#print_summary(post_samples, 0.89, False)


# Note: For an actual application we would rather use MCMC, the following line would achive this 
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(modelDiff)
mcmc = MCMC(kernel, num_samples=1000, num_warmup=1000, num_chains=1)
mcmc.run(rng_key_,G=G,DiffEI=DiffEI,DiffStdI=DiffStdI)
azMCMC = az.from_numpyro(mcmc)
post_samples = mcmc.get_samples()
mcmc.print_summary()

***
<div class="alert alert-block alert-success">

__*Task 4*: Inspect dimensions of posterior samples__ (5 Points)
</div>

2a) Use the cell below to print the shape of all the elements in the dictionary ```post_samples```.  (2 Points)

*For a): Implement your answer using the cell below*

b) Explain in your own words, for every element in the dictionary ```post_samples```  why it has the respective shape, i.e., define verbally the dimension of each variable. (2 Points)

For example assume a ```a.shape = (3,4)``` you should say what the 3 and the 4 mean. 

*For b): Add your answer here:*

YOUR ANSWER HERE

c) Explain in your own words why ```post_samples``` does not include ```DiffEI``` and ```DiffStdI```, as it was the case for ```prior_samples``` above. (1 Point)

YOUR ANSWER HERE


In [ ]:
# Add your solution to 4a)
# Print the shape of elements
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
az.plot_posterior(azMCMC,var_names=['mu_DiffEI']);

In [ ]:
az.plot_posterior(azMCMC,var_names=['mu_DiffStdI']);

In [ ]:
az.plot_posterior(azMCMC,var_names=['sigma_DiffEI']);

In [ ]:
az.plot_posterior(azMCMC,var_names=['sigma_DiffStdI']);

***
<div class="alert alert-block alert-success">

__*Task 5*: Check if inference worked__ (3 Points)

Discuss how you could check if inference has worked. Based on the results and plots above list clearly three aspects you would consider to check if inference has worked. 
</div>

YOUR ANSWER HERE


# Prepare outputs to test hypothesis 

The following plots are useful to use the posterior samples to test (some of) our hypotheses. Note that this is still based on the synthetic data where we know what the true values are. So they cannot actually be used to test our hypothesis. But we can check, using data for which we know that our hypotheses are true, how we could prepare plots to test the hypothesis. The idea is that once we work with the actual data we will use exactly the same types of plots.  

Note that the plots below are intentionally without axis descriptions, it is part of the task to figure this out. Of course, for an actual application, we would add meaningful axis descriptions. 

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, tight_layout=True,figsize=(8,4))
# Consider this plot for 6a)
ax1.hist((post_samples['mu_DiffEI'][:,1]-post_samples['mu_DiffEI'][:,2]),bins=50);
ax1.set_title('Plot for task 6a')
ax1.set_xlabel('x-label')
ax1.set_ylabel('y-label')

# Consider this plot for 6b)
ax2.hist((post_samples['mu_DiffStdI'][:,1]-post_samples['mu_DiffStdI'][:,0]),bins=50);
ax2.set_title('Plot for task 6b')
ax2.set_xlabel('x-label')
ax2.set_ylabel('y-label')


***
<div class="alert alert-block alert-success">

__*Task 6*: Interpret outputs to test hypothesis__ (6 Points)

</div>

6a) Try to understand what the left plot is showing. (3 points)

- Describe in words what the plot shows

- Discuss which parts of the hypothesis you could test with this plot. To make your answer precise, use the mathematical notation used above to define the hypothesis.

- Describe how you would use the plot to test this part of the hypothesis 

YOUR ANSWER HERE

6b) Try to understand what the right plot is showing. (3 points)

- Describe in words what the plot shows

- Discuss which parts of the hypothesis you could test with this plot. To make your answer precise, use the mathematical notation used above to define the hypothesis.

- Describe how you would use the plot to test this part of the hypothesis 

YOUR ANSWER HERE


